In [1]:
import pandas as pd

In [4]:
tweets = pd.read_csv("cities_tweets_hashtags.csv")

/var/folders/w2/999xw7715kj3_sqxrf0t3nb00000gn/T/ipykernel_68985/1423351188.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv("cities_tweets_hashtags.csv")


In [5]:
tweets.head()

,Unnamed: 0,created_at,id,conversation_id,text,edit_history_tweet_ids,author_id,geo.place_id,referenced_tweets,city,inside,withheld.copyright,withheld.country_codes,withheld.scope
0,0,2021-01-27T06:05:44.000Z,1354309595733381121,1354309595733381121,"Accident, right lane blocked in #FortWorth on ...",['1354309595733381121'],41679815,42e46bc3663a4b5f,NaN,Fort Worth,1,NaN,NaN,NaN
1,1,2021-01-27T01:30:44.000Z,1354240391030185988,1354240391030185988,Accident cleared in #FortWorth on 20 (Ft. Wort...,['1354240391030185988'],41679815,42e46bc3663a4b5f,NaN,Fort Worth,1,NaN,NaN,NaN
2,2,2021-01-27T01:25:43.000Z,1354239127630340097,1354239127630340097,Accident. only the right lane gets through in ...,['1354239127630340097'],41679815,42e46bc3663a4b5f,NaN,Fort Worth,1,NaN,NaN,NaN
3,3,2021-01-27T01:15:44.000Z,1354236615569387521,1354236615569387521,"Accident, right lane blocked in #FortWorth on ...",['1354236615569387521'],41679815,42e46bc3663a4b5f,NaN,Fort Worth,1,NaN,NaN,NaN
4,4,2021-01-27T01:10:43.000Z,1354235354036002816,1354235354036002816,Debris blocks three right lanes in #FortWorth ...,['1354235354036002816'],41679815,42e46bc3663a4b5f,NaN,Fort Worth,1,NaN,NaN,NaN


In [6]:
len(tweets)

79183

In [7]:
#Check for duplicates
tweets.duplicated(subset=["id"]).sum()

34745

In [12]:
tweets_no_dupes = tweets.drop_duplicates(subset=['id'], keep=False)

In [13]:
#ensure we have more than enough
tweets_no_dupes.groupby(by=['city', 'inside'])['id'].count()

city           inside
Boston         0          801
               1         2247
Chicago        0         1597
               1         2631
Fort Worth     0           39
               1         2175
Jacksonville   0          275
               1           17
Las Vegas      0         1044
               1         3315
New York       0         2552
               1         2747
Phoenix        0          507
               1         1995
San Antonio    0          161
               1         1959
San Francisco  0          799
               1         2305
Washington DC  0          146
               1         1549
Name: id, dtype: int64

In [8]:
#tweets_no_dupes as separate file
tweets_no_dupes.drop(columns=["Unnamed: 0"]).to_csv("tweets_no_duplicates.csv")

In [9]:
tweets_no_media_only = tweets_no_dupes.loc[~tweets_no_dupes.text.str.lower().str.contains("just posted a")]

In [10]:
tweets_no_media_only.groupby(by=['city', 'inside'])['id'].count()

city           inside
Boston         0         2703
               1         2091
Chicago        0         2778
               1         2219
Fort Worth     0         1757
               1         1549
Jacksonville   0         2244
               1         1317
Las Vegas      0         2585
               1         1922
New York       0         2379
               1         1808
Phoenix        0         2645
               1         2096
San Antonio    0         2290
               1         1992
San Francisco  0         2577
               1         2079
Washington DC  0         2312
               1         2098
Name: id, dtype: int64

In [13]:
tweets_no_media_only.to_csv("tweets_no_media.csv")

In [11]:
long_tweets_only = tweets_no_media_only.loc[~(tweets_no_media_only.text.str.lower().str.len() < 64)]

In [12]:
long_tweets_only.groupby(by=['city', 'inside'])['id'].count()

city           inside
Boston         0         2118
               1         1187
Chicago        0         2174
               1         1870
Fort Worth     0         1495
               1         1346
Jacksonville   0         1935
               1         1246
Las Vegas      0         2217
               1         1745
New York       0         1945
               1         1539
Phoenix        0         2224
               1         1807
San Antonio    0         1954
               1         1784
San Francisco  0         2360
               1         1925
Washington DC  0         2214
               1         2042
Name: id, dtype: int64

In [15]:
#get samples of 1000 per type
seed = 12345
samples = []
for city in long_tweets_only.city.unique().tolist():
    samples.append(long_tweets_only[(long_tweets_only.city == city) & (long_tweets_only.inside == 1)].sample(n=1000, random_state=seed))
    samples.append(long_tweets_only[(long_tweets_only.city == city) & (long_tweets_only.inside == 0)].sample(n=1000, random_state=seed))

In [16]:
#merge samples into 1 df
sample = pd.concat(samples)

In [17]:
#make sure that worked
sample.groupby(by=['city', 'inside'])['id'].count()

city           inside
Boston         0         1000
               1         1000
Chicago        0         1000
               1         1000
Fort Worth     0         1000
               1         1000
Jacksonville   0         1000
               1         1000
Las Vegas      0         1000
               1         1000
New York       0         1000
               1         1000
Phoenix        0         1000
               1         1000
San Antonio    0         1000
               1         1000
San Francisco  0         1000
               1         1000
Washington DC  0         1000
               1         1000
Name: id, dtype: int64

In [22]:
sample.drop(columns=["Unnamed: 0"]).to_csv("tweets_sample.csv")

In [23]:
sample.loc[sample.text.str.len() < 64].sample(10).text

36159    St. Pires en White Plains, New York https://t....
19971           The Las Vegas Golden Knights are for sluts
43511    Washington DC JUNE 25 😎 Get your tix: https://...
46429       64 Lawn Street, Boston https://t.co/vrE95kULmp
2254     Work day views @ Fort Worth, Texas https://t.c...
2497     Midway grad doing things in Fort Worth https:/...
49136    Never bent, never fold, never told, long live ...
11993    I'm at Moon Valley in Phoenix, AZ https://t.co...
3724     KofiMania is on fire in Fort Worth!! Yes!!! #MITB
44566    47 Crawford Street, Boston https://t.co/6qEXV8...
Name: text, dtype: object